In [155]:
import pandas as pd
import numpy as np

In [156]:
# Import data
results_tourney = pd.read_csv('WDataFiles_Stage1/WNCAATourneyCompactResults.csv')
results_season = pd.read_csv('WDataFiles_Stage1/WRegularSeasonCompactResults.csv')
seeds = pd.read_csv('WDataFiles_Stage1/WNCAATourneySeeds.csv')

In [157]:
print(results_tourney.shape)
print(results_season.shape)
print(seeds.shape)

(1386, 8)
(107012, 8)
(1408, 3)


In [158]:
results_tourney['fromdata'] = 'tourney'
results_season['fromdata'] = 'season'

In [159]:
def concat_team(df):
    df_new = df.copy()
    df_new['WmatchTeam'] = df_new['WTeamID'].astype(str) + '_' + df_new['LTeamID'].astype(str)
    df_new['LmatchTeam'] = df_new['LTeamID'].astype(str) + '_' + df_new['WTeamID'].astype(str)
    return df_new

In [160]:
results_tourney = concat_team(results_tourney)
results_season = concat_team(results_season)

In [161]:
results_tourney = results_tourney[results_tourney['Season'].isin([2017, 2018, 2019])].reset_index(drop=True)
results_season = results_season[results_season['Season'].isin([2017, 2018, 2019])].reset_index(drop=True)

In [162]:
print(results_tourney.shape)
print(results_season.shape)
print(seeds.shape)

(189, 11)
(15659, 11)
(1408, 3)


In [163]:
results_tourney.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,fromdata,WmatchTeam,LmatchTeam
0,2017,137,3113,73,3277,61,N,0,tourney,3113_3277,3277_3113
1,2017,137,3166,76,3405,49,N,0,tourney,3166_3405,3405_3166
2,2017,137,3177,88,3320,67,N,0,tourney,3177_3320,3320_3177
3,2017,137,3199,87,3442,66,H,0,tourney,3199_3442,3442_3199
4,2017,137,3246,73,3125,70,H,0,tourney,3246_3125,3125_3246


# Seed

In [164]:
results_tourney = pd.merge(results_tourney, seeds, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
results_tourney.rename(columns={'Seed': 'WSeed'}, inplace=True)
results_tourney['WSeed'] = results_tourney['WSeed'].astype(str).str[1:].astype(int)

results_tourney = pd.merge(results_tourney, seeds, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
results_tourney.rename(columns={'Seed': 'LSeed'}, inplace=True)
results_tourney['LSeed'] = results_tourney['LSeed'].astype(str).str[1:].astype(int)

results_tourney['diff_seed'] = results_tourney['WSeed'] - results_tourney['LSeed']

In [165]:
results_season = pd.merge(results_season, seeds, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
results_season.rename(columns={'Seed': 'WSeed'}, inplace=True)
results_season['WSeed'] = results_season['WSeed'].fillna('X15').astype(str).str[1:].astype(int)

results_season = pd.merge(results_season, seeds, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
results_season.rename(columns={'Seed': 'LSeed'}, inplace=True)
results_season['LSeed'] = results_season['LSeed'].fillna('X15').astype(str).str[1:].astype(int)

results_season['diff_seed'] = results_season['WSeed'] - results_season['LSeed']

In [166]:
print(results_tourney.shape)
print(results_season.shape)
print(seeds.shape)

(189, 16)
(15659, 16)
(1408, 3)


# Diff

In [167]:
# Easy for tourney
attack_winner_score_to_merge = results_season.groupby(['Season', 'WTeamID']).agg({'WScore': 'sum'}).reset_index()
attack_winner_score_to_merge.rename(columns={'WScore': 'Wscore_season_attack'}, inplace=True)
results_tourney = pd.merge(results_tourney, attack_winner_score_to_merge, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])

attack_looser_score_to_merge = results_season.groupby(['Season', 'LTeamID']).agg({'LScore': 'sum'}).reset_index()
attack_looser_score_to_merge.rename(columns={'LScore': 'LScore_season_attack'}, inplace=True)
results_tourney = pd.merge(results_tourney, attack_looser_score_to_merge, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])

def_winner_score_to_merge = results_season.groupby(['Season', 'WTeamID']).agg({'LScore': 'sum'}).reset_index()
def_winner_score_to_merge.rename(columns={'LScore': 'Wscore_season_def'}, inplace=True)
results_tourney = pd.merge(results_tourney, def_winner_score_to_merge, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])

def_looser_score_to_merge = results_season.groupby(['Season', 'LTeamID']).agg({'WScore': 'sum'}).reset_index()
def_looser_score_to_merge.rename(columns={'WScore': 'Lscore_season_def'}, inplace=True)
results_tourney = pd.merge(results_tourney, def_looser_score_to_merge, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])

results_tourney['diff_def'] = results_tourney['Wscore_season_def'] / results_tourney['Lscore_season_def']
results_tourney['diff_attack'] = results_tourney['Wscore_season_attack'] /results_tourney['LScore_season_attack']

In [168]:
results_tourney['diff_def'] = results_tourney['diff_def'].fillna(-999)
results_tourney['diff_attack'] = results_tourney['diff_attack'].fillna(-999)

In [ ]:
# Not easy for season
j_wscore = results_season.columns.get_loc('WScore')
j_lscore = results_season.columns.get_loc('LScore')
j_team = results_season.columns.get_loc('WTeamID')
for season in range(np.min(results_season['Season']), np.max(results_season['Season'])+1):
    print(season)
    for team in np.unique(results_season['WTeamID']):
        prev_score_watt = 0
        prev_score_latt = 0
        prev_score_wdef = 0
        prev_score_ldef = 0
        for i in range(0, results_season.shape[0]):
            if (results_season.iat[i, j_team]==team) & (results_season.loc[i, 'Season']==season):
                results_season.loc[i, 'Wscore_season_attack'] = results_season.iat[i, j_wscore] + prev_score_watt
                prev_score_watt = results_season.loc[i, 'Wscore_season_attack']
                
                results_season.loc[i, 'Wscore_season_def'] = results_season.iat[i, j_lscore] + prev_score_wdef
                prev_score_wdef = results_season.loc[i, 'Wscore_season_def']
                
            if (results_season.iat[i, j_team]==team) & (results_season.loc[i, 'Season']==season):
                results_season.loc[i, 'LScore_season_attack'] = results_season.iat[i, j_lscore] + prev_score_latt
                prev_score_latt = results_season.loc[i, 'LScore_season_attack']
                
                results_season.loc[i, 'Lscore_season_def'] = results_season.iat[i, j_wscore] + prev_score_ldef
                prev_score_ldef = results_season.loc[i, 'Lscore_season_def']
                
results_season['diff_def'] = results_season['Wscore_season_def'] / results_season['Lscore_season_def']
results_season['diff_attack'] = results_season['Wscore_season_attack'] / results_season['LScore_season_attack']

2017
2018


In [ ]:
results_season['diff_def'] = results_season['diff_def'].fillna(-999)
results_season['diff_attack'] = results_season['diff_attack'].fillna(-999)

In [ ]:
# Easy for tourney
prev_score_to_merge = results_season.groupby(['Season', 'WmatchTeam']).agg({'result': 'sum'}).reset_index()
prev_score_to_merge.rename(columns={'result': 'nb_win_previous'}, inplace=True)
results_tourney = pd.merge(results_tourney, prev_score_to_merge, how='left', left_on=['Season', 'WmatchTeam'], right_on=['Season', 'WmatchTeam'])
results_tourney['nb_win_previous'] = results_tourney['nb_win_previous'].fillna(0)

In [ ]:
# Easy for tourney
prev_score_to_merge = results_season.groupby(['Season', 'WmatchTeam']).agg({'result': 'sum'}).reset_index()
prev_score_to_merge.rename(columns={'result': 'nb_win_previous'}, inplace=True)
results_season = pd.merge(results_season, prev_score_to_merge, how='left', left_on=['Season', 'WmatchTeam'], right_on=['Season', 'WmatchTeam'])
results_season['nb_win_previous'] = results_season['nb_win_previous'].fillna(0)

In [ ]:
print(results_tourney.shape)
print(results_season.shape)
print(seeds.shape)

In [ ]:
results_tourney.tail(10)

# Convert

In [ ]:
def convert_result(results_origin):
    """ Create a combine data with win and loose """
    results = results_origin.copy()
    results['result'] = 1 # win
    Lresults = results.copy()
    Lresults['result'] = 0 # lose
    Lresults['WTeamID'] = results['LTeamID']
    Lresults['LTeamID'] = results['WTeamID']
    Lresults['WScore'] = results['LScore']
    Lresults['LScore'] = results['WScore']
    Lresults['diff_seed'] = -results['diff_seed']
    Lresults['diff_attack'] = 1 / results['diff_attack']
    Lresults['diff_def'] = 1 / results['diff_def']
    # Lresults['nb_win_previous'] = 1 / results['nb_win_previous']
    Lresults['WLoc'].replace({'H': 'A'}, inplace=True)
    
    Lresults['WmatchTeam'] = Lresults['WTeamID'].astype(str) + '_' + Lresults['LTeamID'].astype(str)
    Lresults['LmatchTeam'] = Lresults['LTeamID'].astype(str) + '_' + Lresults['WTeamID'].astype(str)
    
    results = pd.concat([results, Lresults]).reset_index(drop=True)

    results = results.sort_values(by=['Season', 'DayNum', 'WmatchTeam'], ascending=True)
    results.reset_index(inplace=True, drop=True)
    
    return results

In [ ]:
results_season = convert_result(results_season)
results_tourney = convert_result(results_tourney)

In [ ]:
results = pd.concat([results_season, results_tourney]).reset_index(drop=True)

In [ ]:
np.unique(results['Season'])

# Model

In [ ]:
# Create fold strategy
# To do: weight on fold
myCViterator = []
weight = []
for fold in [2017, 2018, 2019]:
    trainIndices = results[(results['Season']==fold) & (results['fromdata']=='season')].index.values.astype(int)
    testIndices =  results[(results['Season']==fold) & (results['fromdata']=='tourney')].index.values.astype(int)
    myCViterator.append((trainIndices, testIndices))

In [ ]:
variables = ['diff_seed', 'diff_def', 'diff_attack']
X = results[variables]
y = results['result']

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator
class CategoriesHarm(TransformerMixin, BaseEstimator):
    
    def __init__(self):
        self.cat_save = None 
        
    def fit(self, X, y=None):
        
        self.cat_save = []
        for i in range(X.shape[1]):
            self.cat_save.append(np.unique(X.iloc[:, i]))
        return self
        
    def transform(self, X):
        
        X_new = X.copy()
        for i in range(X.shape[1]):
            X_new.iloc[~np.isin(X_new.iloc[:, i], self.cat_save[i]), i] = self.cat_save[i][-1]
        
        return X_new

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import log_loss
from sklearn.metrics import make_scorer


# To do: change to real binner
pip_num = Pipeline([('binnernum', KBinsDiscretizer(n_bins=5, encode='ordinal')),
                    ('onehotnum', OneHotEncoder(drop='first', sparse=False))])
pip_cat = Pipeline([('harmcat', CategoriesHarm()),
                    ('onehotcat', OneHotEncoder(drop='first', sparse=False))])
data_prep = ColumnTransformer([
    ('cat_feat', pip_num, ['diff_attack', 'diff_def']),
    ('num_feat', pip_cat, ['diff_seed'])])

In [ ]:
model = Pipeline([('prep', data_prep),
                  ('model', linear_model.LogisticRegression(C=30))])
for i, cv in enumerate(myCViterator):
    model.fit(X.iloc[cv[0], :], y.iloc[cv[0]])
    proba = model.predict_proba(X.iloc[cv[1], :])[:, 1]
    print(log_loss(y.iloc[cv[1]], proba))

In [ ]:
sub = pd.read_csv('WSampleSubmissionStage2_2019.csv')

In [ ]:
sub['Season'] = sub['ID'].str.split("_", n=1, expand=True)[0].astype('int')
sub['WTeamID'] = sub['ID'].str.split("_", n=2, expand=True)[1].astype('int')
sub['LTeamID'] = sub['ID'].str.split("_", n=3, expand=True)[2].astype('int')
sub = pd.merge(sub, seeds, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
sub.rename(columns={'Seed': 'WSeed'}, inplace=True)
sub['WSeed'] = sub['WSeed'].astype(str).str[1:].astype(int)

sub = pd.merge(sub, seeds, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
sub.rename(columns={'Seed': 'LSeed'}, inplace=True)
sub['LSeed'] = sub['LSeed'].astype(str).str[1:].astype(int)

In [ ]:
sub['diff_seed'] = sub['WSeed'] - sub['LSeed']

In [ ]:
# Easy for tourney
attack_winner_score_to_merge = results_season.groupby(['Season', 'WTeamID']).agg({'WScore': 'sum'}).reset_index()
attack_winner_score_to_merge.rename(columns={'WScore': 'Wscore_season_attack'}, inplace=True)
sub = pd.merge(sub, attack_winner_score_to_merge, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])

attack_looser_score_to_merge = results_season.groupby(['Season', 'LTeamID']).agg({'LScore': 'sum'}).reset_index()
attack_looser_score_to_merge.rename(columns={'LScore': 'LScore_season_attack'}, inplace=True)
sub = pd.merge(sub, attack_looser_score_to_merge, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])

def_winner_score_to_merge = results_season.groupby(['Season', 'WTeamID']).agg({'LScore': 'sum'}).reset_index()
def_winner_score_to_merge.rename(columns={'LScore': 'Wscore_season_def'}, inplace=True)
sub = pd.merge(sub, def_winner_score_to_merge, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'WTeamID'])

def_looser_score_to_merge = results_season.groupby(['Season', 'LTeamID']).agg({'WScore': 'sum'}).reset_index()
def_looser_score_to_merge.rename(columns={'WScore': 'Lscore_season_def'}, inplace=True)
sub = pd.merge(sub, def_looser_score_to_merge, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'LTeamID'])

sub['diff_def'] = sub['Wscore_season_def'] / sub['Lscore_season_def']
sub['diff_attack'] = sub['Wscore_season_attack'] / sub['LScore_season_attack']

In [ ]:
X_sub = sub[variables]
proba = model.predict_proba(X_sub)[:, 1]

In [ ]:
sub = pd.read_csv('WSampleSubmissionStage2_2019.csv')

In [ ]:
sub['Pred'] = proba

In [ ]:
sub.to_csv('sub.csv', index=False)